In [200]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_SecurityCoupon(disclosureIndex):
    
    # SecurityCoupon Sheet
    def european_to_float(value):
        """
        Convert a European formatted string to a float.
        E.g., "30.416.600" -> 30416600.00
            "30.416.600,99" -> 30416600.99
        """
        if isinstance(value, str):
            value = value.replace('.', '').replace(',', '.')
        try:
            return float(value)
        except ValueError:
            return None
    
    # Step 1: Fetch HTML content
    url = f"https://www.kap.org.tr/tr/Bildirim/{disclosureIndex}"
    response = requests.get(url)

    if response.status_code == 200:
        html_content = response.text
        # print(html_content)
    else:
        raise Exception(f"Failed to fetch webpage: {response.status_code}")

    # Step 2: Parse the HTML
    soup = BeautifulSoup(html_content, 'html.parser')
    

    # Extract several params
    paramdict = {}
    for label in ["ISIN Kodu", "Döviz Cinsi", "İhraç Fiyatı", "Faiz Oranı - Yıllık Basit (%)", "Satışı Gerçekleştirilen Nominal Tutar", "Satışa Başlanma Tarihi", "Kupon Sayısı"]:
        param = None
        for row in soup.find_all("tr"):
            label_div = row.find("div", class_="bold font14")
            if label_div and label in label_div.text:
                param = row.find("div", class_="gwt-HTML control-label lineheight-32px").text.strip()
                paramdict[label] = param
                break
    print(paramdict)
    
    # # Extract "Döviz Cinsi"
    # currency = None
    # for row in soup.find_all("tr"):
    #     label_div = row.find("div", class_="bold font14")
    #     if label_div and "Döviz Cinsi" in label_div.text:
    #         currency = row.find("div", class_="gwt-HTML control-label lineheight-32px").text.strip()
    #         break
    
    # print("ISIN Kodu =", isin_code)

    # Step 4: Extract tables
    tables = pd.read_html(str(soup))

    # Step 5: Process and display tables
    print(f"Number of tables found: {len(tables)}")

    df = tables[5]
    new_header = df.iloc[0]  # save first row as column headers row
    df = df.iloc[1:, :]  # Take the data less the header row
    df.columns = new_header  # Set the header row as the df header
    df["ISIN_CODE"] = paramdict["ISIN Kodu"]
    df.set_index(df.iloc[:, -1], inplace=True)
    df["COUPON_DATE"] = pd.to_datetime(df["Ödeme Tarihi"], format="%d.%m.%Y")
    df["Ödeme Tutarı"] = df["Ödeme Tutarı"].apply(european_to_float)
    df["COUPON_RATE"] = df["Faiz Oranı - Dönemsel (%)"].apply(float) / 10000
    df = df.iloc[:, [-2, -1]]
    df_security_coupon = df.dropna()
    
    # Security Sheet
    if paramdict["Döviz Cinsi"] == "TRY":
        basis = "ACTL365"
    elif paramdict["Döviz Cinsi"] == "EUR":
        basis = "EU30360"
    else:
        basis = "US30360"
    
    fis_dict = {
        "ISIN_CODE":paramdict["ISIN Kodu"],
        "INSTRUMENT_TYPE":None,  # hard coded, fix later.
        "MATURITY_DATE":df["COUPON_DATE"][-1],
        "CURRENCY":paramdict["Döviz Cinsi"],
        "FREQUENCY":paramdict["Kupon Sayısı"],
        "COUPON":european_to_float(paramdict["Faiz Oranı - Yıllık Basit (%)"]),
        "SPREAD":0,  # hard coded, fix later.
        "ISSUER_CODE":None,  # hard coded, fix later.
        "ISSUE_INDEX":0,  # hard coded, fix later.
        "ISSUE_DATE":pd.to_datetime(paramdict["Satışa Başlanma Tarihi"], format="%d.%m.%Y"),
        "DAY_YEAR_BASIS":basis,
        "ISSUE_PRICE":paramdict["İhraç Fiyatı"],
        "totalIssuedAmount":european_to_float(paramdict["Satışı Gerçekleştirilen Nominal Tutar"]),
        "securityType":None,  # hard coded, fix later.
        "fundUser":None  # hard coded, fix later.
        }
    
    df_security = pd.Series(fis_dict)
    
    return df_security, df_security_coupon
    

In [201]:
df1, df2 = get_SecurityCoupon(1281578)
df1

{'ISIN Kodu': 'TRSKORT52525', 'Döviz Cinsi': 'TRY', 'İhraç Fiyatı': '1', 'Faiz Oranı - Yıllık Basit (%)': '61', 'Satışı Gerçekleştirilen Nominal Tutar': '200.000.000', 'Satışa Başlanma Tarihi': '02.05.2024', 'Kupon Sayısı': '4'}
Number of tables found: 10


C:\Users\adevr\AppData\Local\Temp\ipykernel_1664\3393452239.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ISIN_CODE"] = paramdict["ISIN Kodu"]
C:\Users\adevr\AppData\Local\Temp\ipykernel_1664\3393452239.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["COUPON_DATE"] = pd.to_datetime(df["Ödeme Tarihi"], format="%d.%m.%Y")
C:\Users\adevr\AppData\Local\Temp\ipykernel_1664\3393452239.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

ISIN_CODE                   TRSKORT52525
INSTRUMENT_TYPE                     None
MATURITY_DATE        2025-05-22 00:00:00
CURRENCY                             TRY
FREQUENCY                              4
COUPON                              61.0
SPREAD                                 0
ISSUER_CODE                         None
ISSUE_INDEX                            0
ISSUE_DATE           2024-05-02 00:00:00
DAY_YEAR_BASIS                   ACTL365
ISSUE_PRICE                            1
totalIssuedAmount            200000000.0
securityType                        None
fundUser                            None
dtype: object

In [202]:
df2

,COUPON_DATE,COUPON_RATE
ISIN_CODE,,
TRSKORT52525,2024-08-05,15.2083
TRSKORT52525,2024-11-04,15.2083
TRSKORT52525,2025-02-03,15.2083
TRSKORT52525,2025-05-22,18.0494


In [203]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Step 1: Fetch HTML content
url = "https://www.kap.org.tr/tr/Bildirim/1281664"
response = requests.get(url)

if response.status_code == 200:
    html_content = response.text
else:
    raise Exception(f"Failed to fetch webpage: {response.status_code}")

# Step 2: Parse the HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Step 3: Extract tables
tables = pd.read_html(str(soup))

# Step 4: Process and display tables
print(f"Number of tables found: {len(tables)}")

for index, table in enumerate(tables):
    print(f"Table {index}")
    print(table.head())


Number of tables found: 9
Table 0
                                                   0  \
0  Özet Bilgi  Finansman Bonosu Kupon Ödemesi Hk....   
1                                         Özet Bilgi   
2                   Yapılan Açıklama Güncelleme mi ?   
3                     Yapılan Açıklama Düzeltme mi ?   
4      Yapılan Açıklama Ertelenmiş Bir Açıklama mı ?   

                                    1    2    3    4    5    6    7  
0                                 NaN  NaN  NaN  NaN  NaN  NaN  NaN  
1  Finansman Bonosu Kupon Ödemesi Hk.  NaN  NaN  NaN  NaN  NaN  NaN  
2                               Hayır  NaN  NaN  NaN  NaN  NaN  NaN  
3                                Evet  NaN  NaN  NaN  NaN  NaN  NaN  
4                               Hayır  NaN  NaN  NaN  NaN  NaN  NaN  
Table 1
                                               0  \
0                                     Özet Bilgi   
1               Yapılan Açıklama Güncelleme mi ?   
2                 Yapılan Açıklama Düzeltme m